In [2]:
from PIL import Image
from transformers.image_utils import load_image
img = load_image("../data/original_data/019/IMG_20240220_174554771.jpg")

In [115]:
import numpy as np
from segment_anything import SamPredictor, sam_model_registry
sam = sam_model_registry["vit_b"](checkpoint="/Users/timothee/Downloads/sam_vit_b_01ec64.pth")
predictor = SamPredictor(sam)
predictor.set_image(np.array(img))
masks, _, _ = predictor.predict()

In [83]:
def get_segmentation(image: Image, predictor: SamPredictor):
    predictor.set_image(np.array(image))
    mask, _, _ = predictor.predict()

    return mask

In [119]:
masked = np.where(~mask[..., None], img_array, 0)

In [120]:
masked

array([[[118, 111,  95],
        [109, 102,  86],
        [107, 101,  85],
        ...,
        [226, 206, 231],
        [224, 204, 229],
        [222, 202, 227]],

       [[114, 107,  91],
        [106,  99,  83],
        [104,  98,  82],
        ...,
        [223, 203, 228],
        [222, 202, 227],
        [221, 201, 226]],

       [[108, 100,  87],
        [105,  97,  84],
        [104,  98,  84],
        ...,
        [223, 203, 228],
        [222, 202, 227],
        [222, 202, 227]],

       ...,

       [[ 31,  26,  46],
        [ 28,  23,  43],
        [ 32,  27,  47],
        ...,
        [ 81,  40,  48],
        [ 76,  35,  43],
        [ 71,  30,  38]],

       [[ 30,  25,  47],
        [ 30,  25,  47],
        [ 33,  28,  50],
        ...,
        [ 85,  40,  47],
        [ 80,  35,  42],
        [ 70,  25,  32]],

       [[ 30,  25,  47],
        [ 32,  27,  49],
        [ 34,  29,  51],
        ...,
        [ 77,  32,  39],
        [ 79,  34,  41],
        [ 76,  31,  38]]

In [116]:
mask = masks[0]  # shape (H, W)
img_array = np.array(img)

# Invert mask if necessary to keep the object and remove background
masked = np.where(~mask[..., None], img_array, 0)  # keep object pixels

Image.fromarray(masked.astype(np.uint8)).show()

In [121]:
mask = masks[0]
img_array = np.array(img)

# 1. Get bounding box
coords = np.argwhere(mask)
print(coords)
ymin, xmin = coords.min(axis=0)
ymax, xmax = coords.max(axis=0) + 1

# 2. Crop
cropped_img = img_array[ymin:ymax, xmin:xmax]
cropped_mask = mask[ymin:ymax, xmin:xmax]

# 3. Apply mask inside crop
final = np.where(~cropped_mask[..., None], cropped_img, 0)

# 4. Resize to classifier input size (e.g. 224x224)
from PIL import Image
final_pil = Image.fromarray(final.astype(np.uint8)).resize((224, 224))

[]


ValueError: zero-size array to reduction operation minimum which has no identity

In [94]:
final_pil.show()

In [122]:
import numpy as np
from PIL import Image

def replace_background(img: Image.Image, mask: np.ndarray, new_bg: Image.Image, size: int = 224) -> Image.Image:
    """
    Crops an object based on its mask and replaces the background with another image.
    
    Args:
        img (PIL.Image): Original input image.
        mask (np.ndarray): Boolean mask (H, W), True where object is.
        new_bg (PIL.Image): Replacement background image.
        size (int): Output image size (size x size).
        
    Returns:
        PIL.Image: Object on new background, resized.
    """
    img_array = np.array(img)
    bg_array = np.array(new_bg.resize(img.size))  # match background to original size
    
    # Ensure background has 3 channels
    if bg_array.ndim == 2:  # grayscale → RGB
        bg_array = np.stack([bg_array]*3, axis=-1)
    
    # Apply mask: object from original, background from new_bg
    composite = np.where(~mask[..., None], img_array, bg_array)
    
    # Crop tightly around the object
    """ coords = np.argwhere(mask)
    ymin, xmin = coords.min(axis=0)
    ymax, xmax = coords.max(axis=0) + 1
    composite_crop = composite[ymin:ymax, xmin:xmax] """
    
    # Resize to classifier input size
    final_pil = Image.fromarray(composite.astype(np.uint8)).resize((size, size))
    
    return final_pil

replace_background(img, mask, img2).show()

In [3]:
import pandas as pd

path = '~/Documents/programmation/unsplash-research-dataset-lite-latest/photos.csv000'
df = pd.read_csv(path, sep='\t', header=0)

In [8]:
df

,photo_id,photo_url,photo_image_url,photo_submitted_at,photo_featured,photo_width,photo_height,photo_aspect_ratio,photo_description,photographer_username,...,photo_location_country,photo_location_city,stats_views,stats_downloads,ai_description,ai_primary_landmark_name,ai_primary_landmark_latitude,ai_primary_landmark_longitude,ai_primary_landmark_confidence,blur_hash
0,oSf8ePoG9NU,https://unsplash.com/photos/oSf8ePoG9NU,https://images.unsplash.com/20/frozen-grass.JPG,2013-12-04 14:58:49,t,4000,2667,1.50,NaN,andrekoch,...,NaN,NaN,4296224,30072,black road in between white and brown grass ac...,NaN,NaN,NaN,NaN,LPBM;~RkM|of~VRkNHof%1RjRkof
1,DlsOa5moK4w,https://unsplash.com/photos/DlsOa5moK4w,https://images.unsplash.com/reserve/dRA4UuMBR2...,2014-07-10 17:19:38,t,5184,3456,1.50,Fresh blueberries,majkmmiklavc,...,NaN,NaN,8731049,26411,selective focus of blueberry plant,NaN,NaN,NaN,NaN,LiJ+M4s=-pj;~qx]RpbXx1ogRkob
2,XBGacbT3vXI,https://unsplash.com/photos/XBGacbT3vXI,https://images.unsplash.com/photo-143633523196...,2015-07-08 05:54:38.486126,t,6000,4000,1.50,NaN,shooshanig,...,NaN,NaN,2224540,19250,photography of calm beach side,NaN,NaN,NaN,NaN,LSL}HBR*a{fP_NWVoej[x^s.j[ay
3,FjikPptEbZg,https://unsplash.com/photos/FjikPptEbZg,https://images.unsplash.com/flagged/photo-1441...,2015-08-31 07:12:27.700568,f,3008,2008,1.50,NaN,websanya,...,France,Saint-Étienne-de-Tinée,45536,421,green trees on mountain under white clouds dur...,NaN,NaN,NaN,NaN,LyF7Cux[M{kB?^t6azbHRjjrt7ae
4,PXdBkNF8rlk,https://unsplash.com/photos/PXdBkNF8rlk,https://images.unsplash.com/photo-144596683821...,2015-10-27 17:30:25.724092,t,3000,2000,1.50,Icebergs of Iceland’s Vatnajokull,andersjilden,...,Iceland,NaN,7394191,43442,icebergs floating on body of water during daytime,NaN,NaN,NaN,NaN,LQG]];?b~qayBtIU9Ft79xj[tRM|
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,3dVjKoVYgD0,https://unsplash.com/photos/3dVjKoVYgD0,https://images.unsplash.com/photo-158788290090...,2020-04-26 06:37:30.314795,t,6124,4082,1.50,NaN,hduu1,...,NaN,NaN,536839,1694,person in black pants and black shoes standing...,NaN,NaN,NaN,NaN,"L57Ko7sS578_0LV@,;%hIUs:s9bw"
24996,EJ79f7Vew8A,https://unsplash.com/photos/EJ79f7Vew8A,https://images.unsplash.com/photo-159052069445...,2020-05-26 19:18:34.167012,t,3000,2002,1.50,NaN,alex_andrews,...,NaN,NaN,560146,15552,green grass field with birds flying during day...,NaN,NaN,NaN,NaN,L|Ih{Taet7j[_4j[j]j[jeofWBj[
24997,HlY0e1K5zQ4,https://unsplash.com/photos/HlY0e1K5zQ4,https://images.unsplash.com/photo-159036566592...,2020-05-25 00:14:37.331858,t,4160,6240,0.67,NaN,ymoran,...,NaN,NaN,342418,1169,brown wooden house near trees during daytime,NaN,NaN,NaN,NaN,LbGIo#%M-:WB~q%MtRj@WBofRjWB
24998,fJQamCZIZf8,https://unsplash.com/photos/fJQamCZIZf8,https://images.unsplash.com/photo-158982450375...,2020-05-18 17:56:09.252433,t,3078,5472,0.56,NaN,danigarcia,...,United States,Page,1643619,16984,brown and white cave with light,NaN,NaN,NaN,NaN,LNIx2UJQ0|njADNb-VX7FxJRsAR*


In [ ]:
from tqdm import tqdm
import numpy as np
import torch
from torchvision.transforms import v2
from torch.utils.data import Dataset, DataLoader
from src.data import make_transform
from src.utils import load_config
from transformers import AutoImageProcessor, AutoModel

#segmentation_config = load_config("")

class BackgroundDataset(Dataset):
    def __init__(self, urls: pd.Series, transforms: v2):
        self.urls = urls
        self.transforms = transforms

    def __len__(self):
        return self.urls.size
    
    def __getitem__(self, idx):
        return self.transforms(load_image(self.urls[idx]))

batch_size = 8
batches = 4
backgrounds_num = batch_size * batches
background_dataset = BackgroundDataset(df[:backgrounds_num]["photo_image_url"], make_transform())
background_dataloader = DataLoader(background_dataset, batch_size=batch_size)
processor = AutoImageProcessor.from_pretrained("facebook/dinov3-vits16-pretrain-lvd1689m")
model = AutoModel.from_pretrained("facebook/dinov3-vits16-pretrain-lvd1689m", dtype=torch.float32)

def get_background_embeddings(processor: AutoImageProcessor, model: AutoModel, background_dataloader: DataLoader) -> np.ndarray:
    all_embeddings = []
    for image_batch in tqdm(background_dataloader):
        inputs = processor(images=image_batch, return_tensors="pt")
        outputs = model(**inputs)
        embeddings = outputs.pooler_output if hasattr(outputs, 'pooler_output') else outputs.last_hidden_state.mean(dim=1)
        all_embeddings.append(embeddings)
    
    return torch.stack(all_embeddings).flatten(0, 1).detach().numpy()

bg_embeddings = get_background_embeddings(processor, model, background_dataloader)


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 4/4 [00:23<00:00,  5.87s/it]


RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.

In [ ]:
bg_embeddings

tensor([[0.6154, 0.8896, 0.6784],
        [0.7183, 0.6693, 0.4947]])
tensor([[0.0397, 0.3950, 0.7675],
        [0.6109, 0.9429, 0.7557]])
tensor([[0.6154, 0.8896, 0.6784],
        [0.7183, 0.6693, 0.4947],
        [0.0397, 0.3950, 0.7675],
        [0.6109, 0.9429, 0.7557]])


In [80]:
replace_background(img, mask, load_image(df["photo_image_url"][0])).save("./cool.jpg")

In [102]:
import random
random.randint(0, len(df)-1)

17497

In [125]:
from pathlib import Path

base_path = Path("data")
new_source_path = Path("augmented_data16")
image_num_multiplier = 4
base_image_num = 4

for cap_dir in base_path.iterdir():
    label = cap_dir.as_posix().split("/")[-1]
    subpath = new_source_path.joinpath(label)
    if cap_dir.is_dir() and (not subpath.exists() or sum([1 for _ in subpath.iterdir()]) < image_num_multiplier*base_image_num):
        new_source_path.joinpath(label).mkdir(exist_ok=True)
        for image_path in cap_dir.iterdir():
            print(image_path)
            image_name = image_path.as_posix().split("/")[-1]
            image = load_image(image_path.as_posix())
            mask = get_segmentation(np.array(image), predictor)[0]
            for n in range(image_num_multiplier):
                seed = random.randint(0, len(df)-1)
                background_image = load_image(df["photo_image_url"][seed])
                augmented_image = replace_background(image, mask, background_image)
                augmented_image.save(subpath.joinpath('-'.join([str(n), image_name])).as_posix())

data/102/IMG_20240220_214944675.jpg
data/102/IMG_20240220_225319499.jpg
data/102/IMG_20240220_174237969.jpg
data/102/IMG_20240220_171422648.jpg
data/024/IMG_20240220_224257510.jpg
data/024/IMG_20240220_171311334.jpg
data/024/IMG_20240220_220421848.jpg
data/024/IMG_20240220_175440002.jpg
data/023/IMG_20240220_225028497.jpg
data/023/IMG_20240220_215723601.jpg
data/023/IMG_20240220_175621292.jpg
data/023/IMG_20240220_171305531.jpg
data/015/IMG_20240220_175042472.jpg
data/015/IMG_20240220_220808999.jpg
data/015/IMG_20240220_171138085.jpg
data/015/IMG_20240220_224309610.jpg
data/012/IMG_20240220_224514788.jpg
data/012/IMG_20240220_174428896.jpg
data/012/IMG_20240220_220635060.jpg
data/012/IMG_20240220_171113946.jpg
data/079/IMG_20240220_224416681.jpg
data/079/IMG_20240220_172711051.jpg
data/079/IMG_20240220_174739839.jpg
data/079/IMG_20240220_221224954.jpg
data/046/IMG_20240220_171819159.jpg
data/046/IMG_20240220_174307607.jpg
data/046/IMG_20240220_224406695.jpg
data/046/IMG_20240220_221312